In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
from datetime import datetime

### Links to scrape

In [ ]:
links = [
    "https://racing.hkjc.com/racing/information/English/racing/Entries.aspx",
    "https://racing.hkjc.com/racing/information/English/Horse/ListByLocation.aspx?Location=HK",
    "https://racing.hkjc.com/racing/information/English/Jockey/JockeyRanking.aspx",
    "https://racing.hkjc.com/racing/information/English/Jockey/JockeyPastRec.aspx?JockeyId=PZ&Season=Current",
]

### From the subNav menu get all the links

In [14]:
def extract_menu_links(url, menu_rel=None):
    """Extract links with optional filtering by the 'rel' attribute of the menu section's <a> tag"""
    html_content = requests.get(url).content
    soup = BeautifulSoup(html_content, "html.parser")
    links = []

    def process_menu(menu_div):
        nonlocal links
        for dl in menu_div.find_all("dl"):
            for dt in dl.find_all("dt"):
                # If dt.find returns any result
                if a := dt.find("a", href=True):
                    href = a["href"]
                    if href and not href.startswith(("javascript:", "mailto:")):
                        absolute_url = urljoin(url, href)
                        links.append(absolute_url)
                if submenu := dt.find("div", class_="pullmenu"):
                    process_menu(submenu)

    # Find target menu section using rel attribute
    if menu_rel:
        menu_header = soup.find("a", rel=menu_rel)
        if not menu_header:
            return []
        parent_li = menu_header.find_parent("li")
        menu_div = parent_li.find("div", class_="pullmenu") if parent_li else None
        if menu_div:
            process_menu(menu_div)
    else:
        for nav in soup.find_all("div", class_="subNav"):
            process_menu(nav)

    return list(set(links))


target = "https://racing.hkjc.com/racing/english/index.aspx"
rel = "racing-info"
all_links = extract_menu_links(target, menu_rel=rel)
all_links


['https://racing.hkjc.com/racing/information/English/Racing/JKCScheduledRides.aspx',
 'https://racing.hkjc.com/racing/information/English/Racing/Localtrackwork.aspx',
 'https://racing.hkjc.com/racing/english/racing-info/newhorse.asp',
 'https://racing.hkjc.com/racing/english/racing-info/racing_course_time.aspx',
 'https://racing.hkjc.com/racing/information/english/Racing/summary.aspx',
 'https://racing.hkjc.com/racing/information/English/Reports/RaceReportFull.aspx',
 'https://racing.hkjc.com/racing/information/English/Racing/ExceptionalFactors.aspx',
 'https://racing.hkjc.com/racing/information/English/tnc/tncStat.aspx',
 'https://member.hkjc.com/member/english/horse-owner/list-of-bloodstock-agents.aspx',
 'https://racing.hkjc.com/racing/information/english/Horse/HorseFormerName.aspx',
 'https://racing.hkjc.com/racing/information/English/Racing/TNCEntries.aspx',
 'https://campaign.hkjc.com/en/racing/conghua-movement-records.aspx',
 'https://racing.hkjc.com/racing/english/racing-info/r

### Test: Parsing tables from link.

In [ ]:
url = "https://racing.hkjc.com/racing/information/English/racing/Draw.aspx"
url_get = requests.get(url)


def scrape_racing_tables(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    search_results = soup.find_all("div", class_="searchResult")

    all_data = []

    for result in search_results:
        table = result.find("table", class_="table_bd")
        if not table:
            continue

        # Extract race header information
        header = table.find("tr", class_="bg_blue")
        race_info = header.td.text.strip() if header else "Unknown Race"

        for row in table.tbody.find_all("tr"):
            cols = row.find_all("td")
            if len(cols) >= 11:
                all_data.append(
                    {
                        "Race": race_info,
                        "Draw": cols[0].text.strip(),
                        "Runners": cols[1].text.strip(),
                        "Win": cols[2].text.strip(),
                        "Second": cols[3].text.strip(),
                        "Third": cols[4].text.strip(),
                        "Fourth": cols[5].text.strip(),
                        "Win%": cols[6].text.strip(),
                        "Place%": cols[7].text.strip(),
                        "Show%": cols[8].text.strip(),
                        "Fourth%": cols[9].text.strip(),
                    }
                )

    return pd.DataFrame(all_data)


scrape_racing_tables(url_get.content)

### Horse Info

In [49]:
def scrape_horse_info_links(url: str):
    html_content = requests.get(url).content
    soup = BeautifulSoup(html_content, "html.parser")
    cells = soup.find_all("td", class_="table_eng_text")

    hrefs = []

    for cell in cells:
        links = cell.find_all("a", {"class": "table_eng_text"})
        for link in links:
            href = link.get("href")
            if href:
                hrefs.append("https://racing.hkjc.com" + href)
    return hrefs


# links = scrape_horse_info_links(
#     "https://racing.hkjc.com/racing/information/english/Horse/ListByLocation.aspx?Location=HK"
# )

# TODO: Rewrite the below crap, too convoluted.


def parse_horse_profile(url):
    html_content = requests.get(url).content
    soup = BeautifulSoup(html_content, "html.parser")
    profile = {"form_links": {}}

    # Name extraction with error handling
    if title := soup.find("span", {"class": "title_text"}):
        profile["name"] = title.get_text(strip=True)

    # Form links parsing
    if left_col := soup.find("td", class_="table_eng_text"):
        profile["form_links"] = {
            link.text.strip(): f"https://racing.hkjc.com{link['href']}"
            for link in left_col.find_all("a", href=True)
            if not link["href"].startswith("javascript")
        }

    # Main data parsing
    for table in soup.find_all("table", class_="table_eng_text"):
        for row in table.find_all("tr"):
            cells = row.find_all("td")

            key = cells[0].get_text(strip=True).rstrip(":*")
            value_cell = cells[2]

            # Handle links
            # In the main parsing loop:
            # Handle dropdowns FIRST
            if select := value_cell.find("select"):
                profile[key] = [
                    opt.get_text(strip=True) for opt in select.find_all("option")
                ]
                continue  # Skip link processing for this cell

            # Then handle links (but skip javascript links)
            if link := value_cell.find(
                "a", href=lambda h: h and not h.startswith("javascript")
            ):
                profile[f"{key} Text"] = link.get_text(strip=True)
                profile[f"{key} URL"] = f"https://racing.hkjc.com{link['href']}"
                continue

            # Extract and clean value
            raw_value = value_cell.get_text(separator=" ", strip=True)

            # Date handling
            if "(" in raw_value and ")" in raw_value:
                location, date_part = raw_value.split("(", 1)
                date_str = date_part.split(")")[0].strip()
                try:
                    date = datetime.strptime(date_str, "%d/%m/%Y").date().isoformat()
                except ValueError:
                    date = date_str
                profile["Current Stable Location"] = location.strip()
                profile["Arrival Date"] = date
                continue

            # Numeric conversion
            if raw_value.replace("$", "").replace(",", "").isdigit():
                value = int(raw_value.replace("$", "").replace(",", ""))
            elif "-" in raw_value and len(raw_value.split("-")) == 4:
                value = list(map(int, raw_value.split("-")))
            else:
                value = raw_value

            # Split compound keys
            if "/" in key:
                parts = key.split("/")
                values = value.split("/") if isinstance(value, str) else [value]
                for p, v in zip(parts, values):
                    profile[p.strip()] = v.strip() if isinstance(v, str) else v
            else:
                profile[key] = value

    return profile


parse_horse_profile(
    "https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K079"
)


{'form_links': {'Form Records': 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K079',
  'Rating/Wt/Placing': 'https://racing.hkjc.com/racing/information/English/Horse/RatingResultWeight.aspx?HorseId=HK_2024_K079',
  'Performance by Distance': 'https://racing.hkjc.com/racing/information/English/Horse/Performance.aspx?HorseId=HK_2024_K079',
  'Trackwork Records': 'https://racing.hkjc.com/racing/information/English/Trackwork/TrackworkResult.aspx?HorseId=HK_2024_K079',
  'Veterinary Records': 'https://racing.hkjc.com/racing/information/English/VeterinaryRecords/OveHorse.aspx?HorseId=HK_2024_K079',
  'Movement Records': 'https://racing.hkjc.com/racing/information/English/Horse/MovementRecords.aspx?HorseId=HK_2024_K079',
  'Overseas formrecords': 'https://racing.hkjc.com/racing/content/english/pp_formsheet/fse_K079.pdf',
  'Other Horses': 'https://racing.hkjc.com/racing/information/English/Horse/SelectHorse.aspx'},
 'name': 'KINGDOM OF RICHES (K079)',
 '